In [3]:
import os
import pickle
import pandas as pd
import psycopg
from dotenv import dotenv_values
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer


config = dotenv_values(".env")

In [4]:
texts = []
article_names = []

with psycopg.connect(config["PG_URL"]) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT name, content FROM unusual_search.articles")
        for article in cur.fetchall():
            texts.append(article[1])
            article_names.append(article[0])


In [5]:
if os.path.exists('vectorizer.pkl'):
    with open('vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)
else:
    vectorizer = TfidfVectorizer(stop_words='english', min_df=10)
    vectorizer.fit(texts)
    with open('vectorizer.pkl', 'wb') as f:
        pickle.dump(vectorizer, f)


In [6]:
vecs = vectorizer.transform(texts)

In [7]:
df = pd.DataFrame(
    vecs.todense(), columns=vectorizer.get_feature_names_out(), index=article_names)

print(df.head())

                                  00      000  001  002  004  007   01  017  \
52-hertz_whale               0.00000  0.00000  0.0  0.0  0.0  0.0  0.0  0.0   
Bahia_incident               0.02132  0.00000  0.0  0.0  0.0  0.0  0.0  0.0   
Arm-Fall-Off-Boy             0.00000  0.00589  0.0  0.0  0.0  0.0  0.0  0.0   
Arrhichion                   0.00000  0.00000  0.0  0.0  0.0  0.0  0.0  0.0   
Boston_Typewriter_Orchestra  0.00000  0.00000  0.0  0.0  0.0  0.0  0.0  0.0   

                              02  025  ...  zoologist      zoom  zoos   zu  \
52-hertz_whale               0.0  0.0  ...        0.0  0.000000   0.0  0.0   
Bahia_incident               0.0  0.0  ...        0.0  0.000000   0.0  0.0   
Arm-Fall-Off-Boy             0.0  0.0  ...        0.0  0.008302   0.0  0.0   
Arrhichion                   0.0  0.0  ...        0.0  0.000000   0.0  0.0   
Boston_Typewriter_Orchestra  0.0  0.0  ...        0.0  0.000000   0.0  0.0   

                             zur  zürich  émile  état  ü

In [8]:
if os.path.exists('pca.pkl'):
    with open('pca.pkl', 'rb') as f:
        pca = pickle.load(f)
else:
    pca = PCA(n_components=0.95)

    pca.fit(vecs.toarray())

    with open('pca.pkl', 'wb') as f:
        pickle.dump(pca, f)

In [9]:
text_vecs = pca.transform(vecs.toarray())


In [10]:
pca_df = pd.DataFrame(text_vecs, columns=[
                      f'PCA{i}' for i in range(
                          text_vecs.shape[1]
                      )], index=article_names)
print(pca_df.head())

                                 PCA0      PCA1      PCA2      PCA3      PCA4  \
52-hertz_whale               0.011716  0.000625 -0.076410  0.085735 -0.013533   
Bahia_incident              -0.047358  0.035676  0.078935 -0.053133 -0.000148   
Arm-Fall-Off-Boy             0.027157  0.071954 -0.037769 -0.091348 -0.080052   
Arrhichion                   0.002715 -0.034423 -0.016036 -0.029755 -0.018316   
Boston_Typewriter_Orchestra  0.021261  0.137421 -0.045881  0.040956  0.038937   

                                 PCA5      PCA6      PCA7      PCA8      PCA9  \
52-hertz_whale              -0.035069  0.002725  0.005831 -0.013583  0.002069   
Bahia_incident              -0.005015 -0.067471 -0.081275 -0.047213  0.052507   
Arm-Fall-Off-Boy             0.068720  0.040147 -0.014021 -0.112190 -0.021576   
Arrhichion                  -0.015143  0.004982  0.002989 -0.008746  0.010289   
Boston_Typewriter_Orchestra -0.022230  0.060071  0.033001 -0.018949 -0.033331   

                          

In [ ]:
with psycopg.connect(config["PG_URL"]) as conn:
    with conn.cursor() as cur:
        for i, article in enumerate(article_names):
            plain_list = text_vecs[i].tolist()
            cur.execute(
                """
                INSERT INTO unusual_search.embeddings (article_name, embedding_pca)
                VALUES (%s, %s) ON CONFLICT (article_name) DO UPDATE SET embedding_tfidf = %s
                """,
                (article, plain_list, plain_list)
            )
            conn.commit()